In [47]:
# 之前已经实现的函数，接下来要进行调用

# 激活函数
def softmax(x):
    """输入一维数组或者二维数组，输入一维返回一维，输入二维返回二维"""
    if x.ndim == 2: # x=[[1,2,3],[2,3,4]]
        x = x.T # x=[[1,2],[2,3],[3,4]]
        x = x - np.max(x,axis=0) # x=x - [3,4] = [[1-3,2-4],[2-3,3-4],[3-3,4-4]] = [[-2,-2],[-1,-1],[0,0]]
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T # 返回二维数组
    # x.ndim == 1 的情况
    x = x-np.max(x)
    return np.exp(x) / np.sum(np.exp(x)) # 返回一维数组比如,ret.shape=(3,)


# 损失函数
def cross_entropy_error(y,t):
    
    """
    可以传入一维(单个数据)或者二维(多个数据)的 y (预测数据)，一维的会在函数内转换为二维
    传入的 t(监督数据) 可以是one-hot的形式，比如 t=[[0,0,1],[0,1,0]]，或者 t=[2,1]
    """

    delta = 1e-7

    if y.ndim == 1:  # y = [1,2,4] t = [0,0,1]
        # 如果 ndim 为 1，为了统一，转换为 2 维
        y = y.reshape(1,y.size)  # [[1,2,4]]
        t = t.reshape(1,t.size)  # [[0,0,1]]

    if t.size == y.size:  # 将 one-hot 形式转换为普通形式 比如 t=[[0,0,1],[0,1,0]] -->  t=[2,1]
        t = t.argmax(axis=1)
    
    batch_size = y.shape[0] # 数据个数 
    # return -np.sum(t*np.log(y + delta)) / batch_size
    return -np.sum(np.log(y[np.arange(batch_size),t])) / batch_size


# 梯度
# 下面两个函数定义了多个数据的梯度，其中每个数据可以是一元或者多元函数
def _numerical_gradient_1d(f,x):
    """
    求一元或者多元函数梯度，传入的 x 是一维数组，代表坐标，浮点数。比如 x = [1.0,2.0] 就是二元函数在 (1,2) 上的点。求的是在这个点上的二元函数的两个方向的偏导
    """
    h = 1e-4

    grad = np.zeros_like(x) # 假如是二元函数，传入变量 x = [3,4]，则现在 grad = [0,0]，grad[0],grad[1] 分别是二元函数的两个变量的梯度

    for idx in range(x.size): # x: [3,4], idx: [0,1]
        tmp_val = x[idx] # tmp_val=x[0]=3
        x[idx] = tmp_val + h # x: [3+h,4]
        fxh1 = f(x) # [3+h,4] 对应的函数值 f
        x[idx] = tmp_val - h
        fxh2 = f(x)
        grad[idx] = (fxh1-fxh2) / (2*h)
        
        x[idx] = tmp_val # 还原x
        
    return grad
    

def numerical_gradient_2d(f,X):
    """2d数组的梯度"""

    if X.ndim == 1:
        return _numerical_gradient_1d(f,X)
    else:
        grad = np.zeros_like(X) # X=[[2,3,4],[1,2,1]], grad=[[0,0,0],[0,0,0]]
        
        for idx, x in enumerate(X): #  x=[2,3,4],[1,2,1], idx=0,1
            grad[idx] = _numerical_gradient_1d(f,x)
        
        return grad


# 梯度下降函数
def gradient_descent(f, init_x, lr=0.01, step_num=300):
    x = init_x # 假设是二元函数，x=[2,2], f=x[0]**2+x[1]**2
    x_history = []

    for i in range(step_num):
        x_history.append(x.copy())
        
        grad = numerical_gradient_2d(f,x)  # grad=[4,4]
        x -= lr * grad  # x = [2,2] - 0.01*[4,4] = [1.96,1.96]

    return x, np.array(x_history)

In [48]:
import numpy as np
# 1. 定义 x t w
x = np.array([0.6, 0.9]) # 定义的 x
t = np.array([0,0,1]) # 正确解标签
W = np.random.randn(2,3) # 权重
W

array([[ 0.94265553,  0.4580235 , -1.60663573],
       [-1.34852288,  1.01747501,  0.37895319]])

In [49]:
# 2. x,w 点积求 y
z = np.dot(x,W)
z

array([-0.64807728,  1.19054161, -0.62292357])

In [50]:
# 3. 用激活函数处理 y
y = softmax(z)
y

array([0.12028887, 0.75635816, 0.12335296])

In [51]:
# 4. 求损失函数
loss = cross_entropy_error(y,t)
loss

2.092705434172959

损失函数loss 随着 W 的改变而改变  
f = lambda W: cross_entropy_error(y_sm,t)  ^1   
等价于  
def f(W):  
----return cross_entropy_error(y_sm,t)

那么求损失函数关于 W 的梯度，那么能找到一个 W，可以使得损失函数最小，这个 W 就是训练出来的合适的 权重系数  
dW = numerical_gradient_2d(f,W)  

但是 ^1 式无法将 W 传入到 f(也就是损失函数)中，因为 W 改变，z，y都没改变，损失函数也就没改变 

In [52]:
class simpleNet():
    def __init__(self):
        self.W = np.random.randn(2,3)

    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t) 
        return loss

In [53]:
# 1. 定义 x, t, w
net = simpleNet()
x = np.array([0.6, 0.9])
t = np.array([0, 0, 1])
print(net.W)

[[ 0.21412802 -0.09228939  0.62688203]
 [ 1.6645847   1.32434296 -0.73495762]]


In [54]:
# 原先的第2,3,4步包含在 simpleNet 中

# 5. 把 W 和 损失函数关联
f = lambda w: net.loss(x,t)

# 等价于下面的 f
def f(w):
    return net.loss(x,t)

In [55]:
# 7. 总结 - 求 net.W
gradient_descent(f, net.W, lr=0.2, step_num=3)

(array([[ 0.02021979, -0.21694878,  0.94544965],
        [ 1.37372236,  1.13735387, -0.25710619]]),
 array([[[ 0.21412802, -0.09228939,  0.62688203],
         [ 1.6645847 ,  1.32434296, -0.73495762]],
 
        [[ 0.14596087, -0.13404754,  0.73680733],
         [ 1.56233398,  1.26170573, -0.57006967]],
 
        [[ 0.08124007, -0.17578973,  0.84327032],
         [ 1.46525277,  1.19909245, -0.41037518]]]))